In [1]:
import os
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\Program Files\\spark-3.3.0-bin-hadoop3'

In [2]:
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [242]:
from pyspark.sql.functions import col, max as max
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType


So here we try to take the data input as xml format with a schema check and we save this data in parquet format because we will be performing lot of columnar aggregation so we are saving and reading it back from parquet format and also we are keeping a backup of the data

mostly in cloud we save such data in aws s3 or azure data lake with a time stamp

In [201]:
Users = "./data/Users.xml"
df_Users = spark.read.format("com.databricks.spark.xml") \
        .option('rowTag', 'row').load(Users).select(
    F.col('_Id').cast('integer').alias("Id"),
    F.col('_Reputation').cast('integer').alias("Reputation"),
    F.col('_CreationDate').cast('timestamp').alias("CreationDate"),
    F.col('_DisplayName').alias("DisplayName"),
    F.col('_WebsiteUrl').alias("WebsiteUrl"),
    F.col('_LastAccessDate').cast('timestamp').alias("LastAccessDate"),
    F.col('_Location').alias("_Location"),  
    F.col('_Views').cast('integer').alias("Views"),
    F.col('_UpVotes').cast('integer').alias("UpVotes"),
    F.col('_DownVotes').cast('integer').alias("DownVotes"),
    F.col('_ProfileImageUrl').alias("ProfileImageUrl"),
    F.col('_AccountId').cast('integer').alias("AccountId")).dropDuplicates()
df_Users.show(10)
df_Users.write.parquet("./jupyter_output/Users.parquet")
df_Users=spark.read.parquet("./jupyter_output/Users.parquet")

+----+----------+--------------------+--------------+--------------------+--------------------+--------------------+-----+-------+---------+--------------------+---------+
|  Id|Reputation|        CreationDate|   DisplayName|          WebsiteUrl|      LastAccessDate|           _Location|Views|UpVotes|DownVotes|     ProfileImageUrl|AccountId|
+----+----------+--------------------+--------------+--------------------+--------------------+--------------------+-----+-------+---------+--------------------+---------+
| 392|     13145|2011-06-07 08:25:...|    tohuwawohu|https://tohuwawoh...|2022-06-04 14:08:...|             Germany|  494|   1551|       46|https://www.grava...|   137956|
| 467|        83|2011-06-13 02:44:...|Gerardo Curiel|     http://gerar.do|2020-07-04 08:27:...|Melbourne, Victor...|    3|      1|        0|https://www.grava...|    28681|
| 836|         1|2011-09-07 22:22:...|           Ben|http://www.benjam...|2011-09-07 22:22:...|     Aachen, Germany|    0|      0|        0|

we try to do some eda like checking the schema than have a view of the data frame understand the colums

In [202]:
df_Users.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Reputation: integer (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- WebsiteUrl: string (nullable = true)
 |-- LastAccessDate: timestamp (nullable = true)
 |-- _Location: string (nullable = true)
 |-- Views: integer (nullable = true)
 |-- UpVotes: integer (nullable = true)
 |-- DownVotes: integer (nullable = true)
 |-- ProfileImageUrl: string (nullable = true)
 |-- AccountId: integer (nullable = true)



In [203]:
badges = "./data/Badges.xml"
df_Badges = spark.read.format("com.databricks.spark.xml") \
        .option('rowTag', 'row') \
        .load(badges).select(F.col('_Id').cast('integer'),
                            F.col('_Name'),
                            F.col('_UserId').cast('integer'),).dropDuplicates()
df_Badges.write.parquet("./jupyter_output/Badges.parquet")
df_Badges=spark.read.parquet("./jupyter_output/Badges.parquet")

In [204]:
posts = "./data/Posts.xml"
df_Posts = spark.read.format("com.databricks.spark.xml") \
        .option('rowTag', 'row') \
        .load(posts).select(F.col('_Id').cast('integer'),
                            F.col('_CreationDate').cast('timestamp'),
                            F.col('_OwnerUserId').cast('integer'),
                            F.col('_CommentCount').cast('integer'),).dropDuplicates()
df_Posts.write.parquet("./jupyter_output/Posts.parquet")
df_Posts=spark.read.parquet("./jupyter_output/Posts.parquet")

In [205]:
comments = "./data/Comments.xml"
df_Comments = spark.read.format("com.databricks.spark.xml") \
        .option('rowTag', 'row') \
        .load(comments).select(F.col('_Id').cast('integer'),
                            F.col('_CreationDate').cast('timestamp'),
                            F.col('_UserId').cast('integer'),).dropDuplicates()
df_Comments.write.parquet("./jupyter_output/Comments.parquet")
df_Comments=spark.read.parquet("./jupyter_output/Comments.parquet")

In [206]:
df_Comments.printSchema()

root
 |-- _Id: integer (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _UserId: integer (nullable = true)



In [207]:
df_Posts.printSchema()



root
 |-- _Id: integer (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _OwnerUserId: integer (nullable = true)
 |-- _CommentCount: integer (nullable = true)



In [208]:
df_Badges.printSchema()

root
 |-- _Id: integer (nullable = true)
 |-- _Name: string (nullable = true)
 |-- _UserId: integer (nullable = true)



here in badages data frame we run some spark function where if the badge name (column - _name) is editor , cleric we generate new columns is_editor , is_critic with values 0 and 1 where 1 stand for it is editor , it is critic on respective columns and 0 for it is not

In [209]:
df_Badges = df_Badges.withColumn('is_Editor', F.when(F.col('_Name') == "Editor", 1).otherwise(0))
df_Badges = df_Badges.withColumn('is_critic', F.when(F.col('_Name') == "Critic", 1).otherwise(0)).dropDuplicates(['_UserId'])
df_Badges = df_Badges.drop("_Id","_Name")
df_Badges.show()

+-------+---------+---------+
|_UserId|is_Editor|is_critic|
+-------+---------+---------+
|  23336|        0|        0|
|  36131|        0|        0|
|  49308|        0|        0|
|   1088|        0|        0|
|  15846|        0|        0|
|    463|        0|        0|
|    833|        0|        0|
|  31983|        0|        0|
|  18800|        0|        0|
|    148|        0|        0|
|   1829|        0|        0|
|   2366|        0|        0|
|   3175|        0|        0|
|  30970|        0|        0|
|  32539|        0|        0|
|  34239|        0|        0|
|  43302|        0|        0|
|  25462|        0|        0|
|  29814|        0|        0|
|  10817|        0|        0|
+-------+---------+---------+
only showing top 20 rows



here average comment per user is counted over the period that the whole period when he was first active to the last active month even if there are months in between this where he wasnt active is also counted as it falls inside the range of first and last active months

In [210]:
df_commentAverageOverPeriod = df_Comments.groupBy("_UserId").agg(F.count("_UserId").alias("count"), F.max("_CreationDate").alias("max_date"), F.min("_CreationDate").alias("min_date")).orderBy(F.col("count").desc())
df_commentAverageOverPeriod = df_commentAverageOverPeriod.withColumn("total_active_period", F.months_between(F.col("max_date"),F.col("min_date")))
df_commentAverageOverPeriod = df_commentAverageOverPeriod.withColumn("total_active_period", F.floor(F.col("total_active_period")).cast(IntegerType())).withColumn("AverageComment_OverPeriod",  F.round((F.col("count") /( F.col("total_active_period")+1)),2))
df_commentAverageOverPeriod = df_commentAverageOverPeriod.dropDuplicates(['_UserId'])
df_commentAverageOverPeriod.show(10)
df_commentAverageOverPeriod = df_commentAverageOverPeriod.drop("count","max_date","min_date","total_active_period")
df_commentAverageOverPeriod.show(10)

+-------+-----+--------------------+--------------------+-------------------+-------------------------+
|_UserId|count|            max_date|            min_date|total_active_period|AverageComment_OverPeriod|
+-------+-----+--------------------+--------------------+-------------------+-------------------------+
|    833|   23|2021-11-02 12:38:...|2011-09-07 10:58:...|                121|                     0.19|
|   3175|    7|2016-01-13 15:48:...|2013-07-24 10:28:...|                 29|                     0.23|
|  36131|    6|2020-02-06 09:51:...|2020-02-05 12:48:...|                  0|                      6.0|
|   1088|    4|2012-01-08 16:37:...|2011-12-30 18:47:...|                  0|                      4.0|
|  22373|    4|2017-03-07 12:04:...|2016-07-04 09:31:...|                  8|                     0.44|
|   1580|    4|2016-05-25 22:28:...|2015-08-27 12:36:...|                  8|                     0.44|
|  43852|    3|2020-05-25 14:06:...|2020-05-25 10:18:...|       

Here we try to fing the average coments over active months and avtive month is counted is he has active in that month

In [211]:
df_commentAverageActive = df_Comments.withColumn("yyyymm", F.date_format(F.col("_CreationDate"), "yyyy MM")).groupBy("_UserId","yyyymm").count()

df_commentAverageActive = df_commentAverageActive.groupBy("_UserId").agg(F.count("_UserId").alias("Months_active"), F.sum("count").alias("total_comments"),F.max("yyyymm").alias("max_date"), F.min("yyyymm").alias("min_date"))

df_commentAverageActive = df_commentAverageActive.withColumn("AverageValue_OverActiveMonths",  F.round((F.col("total_comments") / F.col("Months_active"))).cast(IntegerType()))
df_commentAverageActive = df_commentAverageActive.dropDuplicates(['_UserId'])

df_commentAverageActive.show()
df_commentAverageActive = df_commentAverageActive.drop("total_comments","max_date","min_date","Months_active")
df_commentAverageActive.show()

+-------+-------------+--------------+--------+--------+-----------------------------+
|_UserId|Months_active|total_comments|max_date|min_date|AverageValue_OverActiveMonths|
+-------+-------------+--------------+--------+--------+-----------------------------+
|   null|            0|          3015| 2022 05| 2011 05|                         null|
|     -1|            1|             3| 2021 09| 2021 09|                            3|
|      2|            5|            17| 2012 04| 2011 05|                            3|
|      4|           21|           267| 2020 06| 2011 05|                           13|
|      5|            1|             1| 2011 06| 2011 06|                            1|
|      7|            2|             7| 2011 06| 2011 05|                            4|
|      8|            8|            94| 2017 07| 2011 05|                           12|
|      9|            3|            13| 2011 07| 2011 05|                            4|
|     10|            4|             5| 2016

Here we are counting the sum of posts by user here both question and answers are taken as posts

In [212]:
df_Count_posts = df_Posts.groupBy("_OwnerUserId").agg(F.count("_OwnerUserId").alias("TotalPostsCount"))
df_Count_posts = df_Count_posts.dropDuplicates(['_OwnerUserId'])
df_Count_posts.show()

+------------+---------------+
|_OwnerUserId|TotalPostsCount|
+------------+---------------+
|         148|              6|
|        1088|              5|
|       18911|              1|
|        1580|              2|
|        2366|              1|
|       16386|              1|
|        6357|              3|
|         833|              4|
|       18654|              1|
|        1829|              3|
|        5803|              1|
|       23015|              1|
|        3175|              3|
|        4900|              6|
|       22373|              3|
|       16861|              1|
|       20735|              2|
|        6397|              1|
|        1238|              1|
|       22097|              1|
+------------+---------------+
only showing top 20 rows



this is no of posts by user in last 90 days

In [213]:
df_last90days_posts = df_Posts.filter(F.col('_CreationDate') >= F.date_sub(F.current_date(), 90))
df_last90days_posts = df_last90days_posts.groupBy("_OwnerUserId").agg(F.count("_OwnerUserId").alias("PostsCountLast90Days"))
df_last90days_posts = df_last90days_posts.dropDuplicates(['_OwnerUserId'])
df_last90days_posts.show()

+------------+--------------------+
|_OwnerUserId|PostsCountLast90Days|
+------------+--------------------+
|       53051|                   1|
|       53291|                   1|
|       53300|                   1|
|       34192|                   1|
|       53296|                   1|
|       46642|                   1|
|       53302|                   1|
|       49023|                   1|
|       53278|                   2|
|       53309|                   1|
|       15264|                   1|
|        1487|                   2|
|       22704|                   1|
|       10837|                   1|
|       44591|                   1|
|       52092|                   1|
|       43989|                   2|
|       53305|                   1|
|       25884|                   1|
|       53289|                   1|
+------------+--------------------+
only showing top 20 rows



it is last date where the user created a post last

In [214]:
df_lastcreation_posts = df_Posts.filter(F.col('_CommentCount') >= 1).groupBy("_OwnerUserId").agg(F.max("_CreationDate").alias("Last_creationDate"))
df_lastcreation_posts = df_lastcreation_posts.dropDuplicates(['_OwnerUserId'])
df_lastcreation_posts.show()

+------------+--------------------+
|_OwnerUserId|   Last_creationDate|
+------------+--------------------+
|         148|2011-09-07 21:25:...|
|       18911|2015-11-04 11:59:...|
|        1580|2012-05-07 23:59:...|
|        2366|2014-08-26 15:58:...|
|       16386|2015-06-28 23:08:...|
|        6357|2015-04-14 01:32:...|
|       18654|2015-10-16 22:32:...|
|        1088|2011-12-29 19:08:...|
|        3175|2014-05-14 13:42:...|
|        4900|2013-12-13 22:54:...|
|       22373|2017-03-06 11:48:...|
|        1829|2012-08-08 14:22:...|
|       20735|2016-06-07 17:55:...|
|        6397|2014-05-26 15:42:...|
|       22097|2016-06-10 18:18:...|
|       35982|2019-01-23 22:28:...|
|       35820|2019-01-10 11:59:...|
|       26425|2017-04-01 21:32:...|
|         833|2020-01-13 13:01:...|
|       30970|2017-12-13 15:17:...|
+------------+--------------------+
only showing top 20 rows



now we join all the tabels with the user table and thus enriching with newly genrated colums here we keep a count after each joins we try to drop the duplicates so the there is explosion of rows due to iterative joins

In [215]:
df_Users.count()

36638

In [216]:
df_Users_temp = df_Users.join(df_lastcreation_posts, df_Users.Id == df_lastcreation_posts._OwnerUserId,how='left')
df_Users_temp = df_Users_temp.drop("_OwnerUserId","_UserId")
df_Users_temp.count()

36638

In [217]:

df_Users_temp = df_Users_temp.join(df_Count_posts, df_Users_temp.Id == df_Count_posts._OwnerUserId,how='left')
df_Users_temp = df_Users_temp.drop("_OwnerUserId")
df_Users_temp.count()

36638

In [218]:

df_Users_temp = df_Users_temp.join(df_last90days_posts, df_Users_temp.Id == df_last90days_posts._OwnerUserId,how='left')
df_Users_temp = df_Users_temp.drop("_OwnerUserId")
df_Users_temp.count()

36638

In [219]:
df_Users_temp.show(5)

+---+----------+--------------------+---------------+--------------------+--------------------+--------------------+-----+-------+---------+---------------+---------+--------------------+---------------+--------------------+
| Id|Reputation|        CreationDate|    DisplayName|          WebsiteUrl|      LastAccessDate|           _Location|Views|UpVotes|DownVotes|ProfileImageUrl|AccountId|   Last_creationDate|TotalPostsCount|PostsCountLast90Days|
+---+----------+--------------------+---------------+--------------------+--------------------+--------------------+-----+-------+---------+---------------+---------+--------------------+---------------+--------------------+
|  9|      1075|2011-05-24 21:28:...|        Jemus42|https://blog.jemu...|2022-01-27 16:21:...|     Bremen, Germany|   29|     48|        0|           null|   177798|2011-05-26 14:39:...|             10|                null|
| 85|       101|2011-05-25 01:26:...|          Marco|                null|2011-05-25 01:26:...|     

In [220]:

df_Users_temp = df_Users_temp.join(df_Badges, df_Users_temp.Id == df_Badges._UserId,how='left')
df_Users_temp = df_Users_temp.drop("_UserId")
df_Users_temp.count()

36638

In [221]:
df_Users_temp.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Reputation: integer (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- WebsiteUrl: string (nullable = true)
 |-- LastAccessDate: timestamp (nullable = true)
 |-- _Location: string (nullable = true)
 |-- Views: integer (nullable = true)
 |-- UpVotes: integer (nullable = true)
 |-- DownVotes: integer (nullable = true)
 |-- ProfileImageUrl: string (nullable = true)
 |-- AccountId: integer (nullable = true)
 |-- Last_creationDate: timestamp (nullable = true)
 |-- TotalPostsCount: long (nullable = true)
 |-- PostsCountLast90Days: long (nullable = true)
 |-- is_Editor: integer (nullable = true)
 |-- is_critic: integer (nullable = true)



In [222]:
df_commentAverageOverPeriod.show()

+-------+-------------------------+
|_UserId|AverageComment_OverPeriod|
+-------+-------------------------+
|    833|                     0.19|
|   3175|                     0.23|
|  36131|                      6.0|
|   1088|                      4.0|
|  22373|                     0.44|
|   1580|                     0.44|
|  43852|                      3.0|
|   4900|                      3.0|
|  20735|                      3.0|
|  11141|                      2.0|
|  38868|                      2.0|
|   1238|                     0.15|
|  18654|                      2.0|
|  15846|                      2.0|
|    463|                      1.0|
|  23015|                      1.0|
|    148|                      1.0|
|  35820|                      1.0|
|   5803|                      1.0|
|  28836|                      1.0|
+-------+-------------------------+
only showing top 20 rows



In [223]:
df_Users_temp = df_Users_temp.join(df_commentAverageOverPeriod, df_Users_temp.Id == df_commentAverageOverPeriod._UserId,how='left')
df_Users_temp = df_Users_temp.drop("_UserId")
df_Users_temp.count()

36638

In [224]:
df_Users_temp.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Reputation: integer (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- WebsiteUrl: string (nullable = true)
 |-- LastAccessDate: timestamp (nullable = true)
 |-- _Location: string (nullable = true)
 |-- Views: integer (nullable = true)
 |-- UpVotes: integer (nullable = true)
 |-- DownVotes: integer (nullable = true)
 |-- ProfileImageUrl: string (nullable = true)
 |-- AccountId: integer (nullable = true)
 |-- Last_creationDate: timestamp (nullable = true)
 |-- TotalPostsCount: long (nullable = true)
 |-- PostsCountLast90Days: long (nullable = true)
 |-- is_Editor: integer (nullable = true)
 |-- is_critic: integer (nullable = true)
 |-- AverageComment_OverPeriod: double (nullable = true)



In [225]:
df_commentAverageActive.show()

+-------+-----------------------------+
|_UserId|AverageValue_OverActiveMonths|
+-------+-----------------------------+
|   1580|                            2|
|    833|                            2|
|  35820|                            1|
|  15846|                            2|
|   2366|                            1|
|  43852|                            3|
|  22373|                            2|
|   1088|                            2|
|   3175|                            2|
|  28836|                            1|
|  36131|                            6|
|   4900|                            3|
|  33722|                            1|
|  11141|                            2|
|  38868|                            2|
|  18654|                            2|
|   1238|                            1|
|    463|                            1|
|    148|                            1|
|  47711|                            1|
+-------+-----------------------------+
only showing top 20 rows



In [226]:
df_Users_temp = df_Users_temp.join(df_commentAverageActive, df_Users_temp.Id == df_commentAverageActive._UserId,how='left')
df_Users_temp = df_Users_temp.drop("_UserId")
df_Users_temp.count()

36638

In [227]:
df_Users_temp.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Reputation: integer (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- WebsiteUrl: string (nullable = true)
 |-- LastAccessDate: timestamp (nullable = true)
 |-- _Location: string (nullable = true)
 |-- Views: integer (nullable = true)
 |-- UpVotes: integer (nullable = true)
 |-- DownVotes: integer (nullable = true)
 |-- ProfileImageUrl: string (nullable = true)
 |-- AccountId: integer (nullable = true)
 |-- Last_creationDate: timestamp (nullable = true)
 |-- TotalPostsCount: long (nullable = true)
 |-- PostsCountLast90Days: long (nullable = true)
 |-- is_Editor: integer (nullable = true)
 |-- is_critic: integer (nullable = true)
 |-- AverageComment_OverPeriod: double (nullable = true)
 |-- AverageValue_OverActiveMonths: integer (nullable = true)

